<a id='start'></a>
# Определение факторов, влияющих на успешность компьютерных игр интернет-магазина

Интернет-магазин «Стримчик» продаёт по всему миру компьютерные игры.

**Цель исследования:**
на основе исторических данных о продажах и рейтинге игр выявить закономерности, определяющие успешность игры, с целью планирования дальнейших рекламных кампаний.

**Описание данных:**

-	*Name* — название игры
-	*Platform* — платформа
-	*Year_of_Release* — год выпуска
-	*Genre* — жанр игры
-	*NA_sales* — продажи в Северной Америке (миллионы проданных копий)
-	*EU_sales* — продажи в Европе (миллионы проданных копий)
-	*JP_sales* — продажи в Японии (миллионы проданных копий)
-	*Other_sales* — продажи в других странах (миллионы проданных копий)
-	*Critic_Score* — оценка критиков (максимум 100)
-	*User_Score* — оценка пользователей (максимум 10)
-	*Rating* — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

**План исследования:**
1. [Загрузка данных](#step1)
2. [Предобработка данных](#step2)
3. [Исследовательский анализ данных](#step3)
4. [Формирование портретов пользователей по регионам](#step4)
5. [Статистический анализ данных](#step5)
6. [Выводы исследования](#step6)

<a id='step1'></a>
## 1. Загрузка данных

In [ ]:
# импортируем библиотеки

import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# загрузим файл с данными и посмотрим общую информацию о датасете

try:
    data = pd.read_csv('/datasets/games.csv')
except:
    data = pd.read_csv('C:/Users/Александра/practicum/games.csv')
    
data.head()

In [ ]:
data.info()

In [ ]:
data.isna().sum()

В загруженном датасете имеются **16 715 наблюдений** по **11 переменным**. По переменным *Name*, *Year_of_Release*, *Genre*, *Critic_Score*, *User_Score* и *Rating* имеются пропущенные значения, которые предстоит обработать на следующем этапе исследования.

<a id='step2'></a>
## 2. Предобработка данных

### 2.1. Замена названий столбцов

In [ ]:
# приведём все названия переменных к нижнему регистру

data.columns = data.columns.str.lower()
data.head()

### 2.2. Проверка на наличие дубликатов

In [ ]:
data.duplicated().sum()

**Явных дубликатов в датафрейме не обнаружено.**

Проверим датафрейм на наличие неявных дубликатов по переменным *name*, *platform* и *year_of_release*:

In [ ]:
data[data.duplicated(subset=['name', 'platform', 'year_of_release'], keep=False)]

In [ ]:
data = data.drop_duplicates(subset=['name', 'platform', 'year_of_release'])
data.info()

После проверки на дубликаты в датасете осталось **16 713 записей**.

### 2.3. Замена типа данных

In [ ]:
# изучим тип данных по переменным датафрейма

data.info()

Отметим следующие изменения, которые необходимо произвести:
1. *year_of_release*: float64 -> int64. Причина: год выхода игры - это целочисленное значение.
2. *user_score*: object -> float64. Причина: оценка пользователей - это не строковая, а количественная переменная, измеряющаяся от 0 до 10.

#### *year_of_release*

In [ ]:
# заменим тип данных переменной year_of_release

data['year_of_release'] = data['year_of_release'].astype('Int64')
data.info()

#### *user_score*

In [ ]:
# изучим список уникальных значений по переменной user_score

data['user_score'].unique()

Стало понятно, почему тип данных по переменной *user_score* был определён как *object*: среди значений встречаются не только вещественные числа, но и такое значение, как "tbd". Вероятно, оно расшифровывается как "to be determined", т.е. оценка пользователей для этой игры ещё не была выставлена (например, по причине того, что игра вышла недавно, или опрос пользователей ещё не завершён).

Изучим, сколько таких записей присутствует в выборке.

In [ ]:
print('В выборке наблюдается', data[data['user_score'] == 'tbd']['user_score'].count(), \
      'записи, в которых переменная user_score принимает значение "tbd".')

Проверим, в какие года были выпущены игры, по которым переменная *user_score* принимает значение "tbd".

**Предположение**: игры, которые вышли незадолго до 2016 года, ещё не были оценены пользователями, поэтому переменная *user_score* приняла значение "tbd".

In [ ]:
data[data['user_score'] == 'tbd']['year_of_release'].describe()

In [ ]:
data.loc[data['user_score'] == 'tbd', 'year_of_release'].hist(figsize=(14,7), bins=19, width=0.8)
plt.title('Распределение переменной year_of_release для записей, \n где значение переменной \
user_score равно "tbd"', fontsize=14, pad=10)
plt.xlabel('Год выпуска', fontsize=12, labelpad=10)
plt.xticks(np.arange(1997, 2016, 1))
plt.ylabel('Частота', fontsize=12, labelpad=10)
plt.grid(None)
sns.despine()
plt.show()

Предположение не было подтверждено. Игры, у которых значение *user_score* равно "tbd", могли быть выпущены как в 2000, так и в 2015 году.

Проверим другое **предположение**: такое значение переменная принимает в том случае, если продажи игры были слишком малы для того, чтобы сформировать оценку от пользователей.

In [ ]:
(data.
 pivot_table(
     index='user_score',
     values=['na_sales', 'eu_sales', 'jp_sales', 'other_sales'],
     aggfunc='median'
))

Предположение также не оправдалось. По всей видимости, при заполнении базы имела место ошибка, поэтому с целью дальнейшего анализа заменим все значения "tbd" в переменной *user_score* на пропущенные значения (NaN).

Избавляться от таких данных не стоит, т.к. они составляют ~ 14.5% всей выборки, что будет слишком много для удаления.

In [ ]:
data['user_score'] = data['user_score'].replace('tbd', np.nan)
data['user_score'].unique()

Можно отметить, что значение "tbd" исчезло из списка уникальных значений переменной *user_score*. Заменим тип *object* на *float64*.

In [ ]:
data['user_score'] = data['user_score'].astype('float64')
data['user_score'].describe()

In [ ]:
data.info()

После замены все переменные имеют необходимый тип данных - можно переходить к обработке пропущенных значений.

### 2.4. Обработка пропущенных значений

In [ ]:
# выведем список переменных и количество пропущенных значений по ним

data.isna().sum()

Необходимо обработать пропущенные значения по переменным:
1. *year_of_release*;
2. *critic_score*;
3. *user_score*;
4. *rating*.

По переменным *name* и *genre* всего по 2 пропуска - кроме того, эти столбцы не играют ключевую роль в анализе. Следовательно, мы можем оставить пропущенные значения в этих переменных.

#### *year_of_release*

In [ ]:
# выведем количество пропущенных значений по переменной

data['year_of_release'].isna().sum()

In [ ]:
print('По переменной year_of_release в датасете пропущено',
      data['year_of_release'].isna().sum(), 'значений, что составляет',
      round(data['year_of_release'].isna().sum()/data.shape[0] * 100, 1),
      '% от общей выборки.')

Т.к. доля пропущенных значений по переменной *year_of_release* составляет всего 1.6% от всей выборки, от таких записей можно избавиться.

In [ ]:
data = data.dropna(subset=['year_of_release'])
data['year_of_release'].isna().sum()

#### *critic_score* и *user_score*

Большое количество пропусков данных по оценкам критиков - существенное препятствие для дальнейшего анализа.

In [ ]:
# изучим распределение переменных critic_score и user_score

data['critic_score'].hist(bins=100, figsize=(10, 7))
plt.title('Распределение переменной critic_score', fontsize=14, pad=10)
plt.xlabel('Рейтинг критиков', fontsize=12, labelpad=10)
plt.ylabel('Частота', fontsize=12, labelpad=10)
plt.grid(False)
sns.despine()
plt.show()

In [ ]:
data['user_score'].hist(bins=100, figsize=(10, 7))
plt.title('Распределение переменной user_score', fontsize=14, pad=10)
plt.xlabel('Рейтинг пользователей', fontsize=12, labelpad=10)
plt.ylabel('Частота', fontsize=12, labelpad=10)
plt.grid(False)
sns.despine()
plt.show()

Распределение переменных *critic_score* и *user_score* напоминает нормальное со сдвигом вправо. Проверим, связаны ли эти оценки между собой.

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(data['critic_score'], data['user_score'], alpha=0.3)
plt.title('Диаграмма рассеяния для переменных critic_score и user_score', fontsize=14, pad=10)
plt.xlabel('critic_score', fontsize=12, labelpad=10)
plt.ylabel('user_score', fontsize=12, labelpad=10)
sns.despine()
plt.show()
print()
print(f"Коэффициент корреляции между переменными: {round(data['critic_score'].corr(data['user_score']), 2)}")

Между переменными наблюдается средняя взаимосвязь, что также подтверждается коэффициентом корреляции на уровне 0.58.

Тем не менее, ввиду того, что по переменной *user_score* также наблюдается много пропущенных значений, мы не можем использовать медианные значения этой переменной для заполнения пропущенных значений переменной *critic_score* (и наоборот).

Удалять записи с пропущенными значениями - также не выход, т.к. подобные наблюдения составляют большую долю датафрейма. Поэтому для дальнейшего анализа **оставим пропущенные значения переменных *critic_score* и *user_score***.

#### *rating*

In [ ]:
# изучим уникальные значения переменной rating и количество пропусков по ней

print('Уникальные значения переменной rating:')
print(data['rating'].value_counts())
print()
print('Количество пропущенных значений:', data['rating'].isna().sum())

**Предположение**: возрастной рейтинг игры коррелирует с её жанром.

Проверим это предположение с помощью частотной таблицы.

In [ ]:
crosstab = pd.crosstab(index=data['genre'], columns=data['rating'])
crosstab['sum'] = crosstab.sum(axis=1)
crosstab = crosstab.sort_values(by=['sum'], ascending=False)
cm = sns.light_palette("green", as_cmap=True)
crosstab = crosstab.style.background_gradient(cmap=cm)
crosstab

In [ ]:
data['rating'].corr(data['genre'], method='spearman')

Предположение не подтвердилось: **возрастной рейтинг игры слабо связан с её жанром**.

Удалить все записи, в которых пропущены значения по переменной *rating*, будет ошибкой ввиду потери слишком большой части выборки.

Про рейтинг ESRB известно, что он назначается для игр в США и Канаде, а в Европе и Японии есть другие организации, определяющие рейтинг компьютерной игры. Предположительно, пропуски значений по переменной rating обусловлены тем, что игра была издана не в США и Канаде. Следовательно, на такие значения целесообразно поставить заглушку, которая будет нести в себе эту информацию.

In [ ]:
data['rating'] = data['rating'].fillna('NonNA')
data['rating'].value_counts()

#### Возможные причины появления пропусков

Что могло привести к пропускам значений по переменным *critic_score*, *user_score* и *rating*?

***1. critic_score***

Наличие 8 578 пропусков по переменной *criric_score* (51.3% от всей выборки) может быть обусловлено следующими факторами:
- недостаточное количество оценок от критиков для формирования рейтинга;
- низкие продажи игры - как следствие, непрохождение минимального количества продаж для направления игры на оценку критиков;
- технические ошибки при выгрузке или переносе данных.

***2. user_score***

Из 9 125 пропущенных значений по переменной *user_score* 2 424 ранее принимали значение "tbd" (25.6% от всех пропущенных значений по переменной). Остальные пропуски могли появиться по следующим причинам:
- недостаточное количество оценок от пользователей для формирования рейтинга;
- технические ошибки при выгрузке или переносе данных.

***3. rating***
- рейтинг ESRB назначается для игр в США и Канаде. В Европе и Японии есть свои рейтинги для компьютерных игр (PEGI и CERO). Вероятно, пропуск рейтинга ESRB у игры означает то, что она была издана не в США или Канаде;
- технические ошибки при выгрузке или переносе данных.

### 2.5. Добавление столбцов

In [ ]:
# добавим в датафрейм столбец с суммарными продажами во всех регионах

data['total_sales'] = data.loc[:, ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
data.info()

### Результаты предобработки данных

В результате предобработки данных были проведены следующие действия:
1. названия столбцов приведены к нижнему регистру;
2. проведена проверка данных на наличие явных и неявных дубликатов;
3. заменён тип данных некоторых переменных;
4. обработаны пропущенные значения в переменных. Пропущенные значения удалены или заменены;
5. добавлен столбец с суммарными продажами по играм.

После проведённой предобработки данных можно приступить к исследовательскому анализу.

<a id='step3'></a>
## 3. Исследовательский анализ данных

### 3.1. Изучение года выпуска игр

In [ ]:
# изучим, сколько игр выпускалось в разные годы, и посмотрим ещё раз 
# на гистограмму переменной

print(data['year_of_release'].value_counts().head(10))

data['year_of_release'].hist(figsize=(10, 7), bins=37)
plt.title('Гистограмма распределения переменной year_of_release', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Частота', fontsize=12, labelpad=10)
plt.grid(False)
sns.despine()
plt.show()

Можно отметить, что **наибольшая часть игр была выпущена в период с 2005 по 2011 год, пик выпуска приходится на 2008 год**.

Игры, выпущенные с 1980 по 1990 год, хоть и составляют небольшую часть выборки, могут представлять ценные наблюдения для последующего анализа. Оснований для удаления этих записей нет.

### 3.2. Изменение продаж по платформам

In [ ]:
# изучим, как менялись продажи по платформам

platform_sales_per_year = (data
                             .pivot_table(
                                index='platform',
                                values='total_sales',
                                columns='year_of_release',
                                aggfunc='sum')
                            )
platform_sales_per_year.head(10)

In [ ]:
# визуализируем сводную таблицу

platform_sales_per_year.head(10).T.plot(kind='line', figsize=(15, 7))
plt.title('Ежегодные продажи игр по платформам', fontsize=14, pad=10)
plt.xlabel('Год', size=12, labelpad=10)
plt.ylabel('Ежегодные продажи', size=12, labelpad=10)
plt.legend(title='Платформа')
sns.despine()
plt.show()

По итогам анализа таблицы видна ошибка: какие-то игры на платформе DS, согласно данным, были выпущены в 1985 году (продажи составляют 0.02), однако сама игровая консоль была создана только в 2004 году.

Избавимся от данного наблюдения.

In [ ]:
data.query('year_of_release == 1985 and platform == "DS"')

In [ ]:
data = data.drop(index=15957)
data

In [ ]:
# выберем платформы с наибольшими суммарными продажами

platform_sales = pd.DataFrame(data
                     .groupby('platform')['total_sales']
                     .sum()
                     .sort_values(ascending=False)
                    .reset_index())
platform_sales.head(10)

Первые 6 платформ демонстрируют наибольшие продажи, далее показатели существенно сокращаются. Изучим на графиках, как менялись продажи этих платформ по годам.

In [ ]:
top6_platforms = platform_sales.loc[:5, 'platform']
platform_sales_per_year[platform_sales_per_year.index.isin(top6_platforms)].T.plot(figsize=(10, 5), grid=True)
plt.title('Динамика глобальных продаж на наиболее популярных платформах', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xlim(1993, 2017)
plt.legend(title='Платформа')
sns.despine()
plt.show()

Продажи игр на каждой из топ-6 платформ имеют схожие тренды на графиках: рост, пик и падение. По "пикам" можно определить и характерный срок, за который появляются новые и исчезают старые платформы.

Так, например, судя по графику, продажи игр на платформе **PS** достигли своего пика в 1998 году, следующий пик продаж - игры на платформе **PS2** - был отмечен в 2002 и 2004 годах. Следующий пик - **PS3** - уже в 2011 году. В среднем процесс исчезновения старых платформ и (одновременного) появления новых составляет 5 лет. Для более точного результата рассчитаем медианный срок жизни платформы.

In [ ]:
# рассчитаем медианный срок жизни платформы

platform_life_time = data.pivot_table(
    index='platform', values='year_of_release', aggfunc=['min', 'max'])
platform_life_time['life_time'] = platform_life_time['max'] - platform_life_time['min'] + 1
platform_life_time['life_time'].median()

Как видно из расчётов, **медианный срок жизни платформы составляет 7 лет**. Проверим, насколько отличается этот показатель, если рассчитать его на основе 6 крупнейших платформ.

In [ ]:
platform_life_time = platform_life_time.reset_index()
top_platform_life_time = platform_life_time[platform_life_time['platform'].isin(top6_platforms)]
top_platform_life_time['life_time'].median()

Интересно, что срок жизни топ-6 популярных платформ существенно больше - 11 лет. Вероятно, чем больше игр приобретается на платформе, тем дольше она "держится" на рынке.

### 3.3. Изучение актуального периода продаж

In [ ]:
# проанализируем продажи, которые компания получала за предыдущие годы

data.groupby('year_of_release')['total_sales'].sum().plot(figsize=(10, 5), grid=True)
plt.title('Динамика глобальных продаж с 1980 по 2016 гг.', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

**Наблюдение**: начиная с 2009 года компания демонстрирует резкое сокращение глобальных продаж.

Необходимое условие для определения актуального периода: "данные помогут построить прогноз на 2017 год". Как было определено на предудыщем шаге, средняя "жизнь" платформы составляет 7 лет.

Актуальным периодом для анализа, позволяющим корректно построить прогноз продаж на будущий период, будет примерно половина такого срока. С целью дальнейшего анализа избавимся от всех наблюдений до 2013 года, т.е. в выборку войдут все продажи за период с 2013 по 2016 год - в этом периоде будет 3 полных года (2013-2015 гг.).

In [ ]:
data = data[data['year_of_release'] > 2012]
data.info()

Получили выборку из **2 233 наблюдений**.

### 3.4. Глобальные лидеры по продажам среди платформ

In [ ]:
# определим, какие платформы с 2013 года лидируют в продажах, а также тренды по продажам

sales_pivot = data.pivot_table(
    index='platform',
    values='total_sales',
    columns='year_of_release',
    aggfunc='sum'
)

sales_pivot['total'] = sales_pivot.sum(axis=1)
sales_pivot = sales_pivot.sort_values(by='total', ascending=False)
sales_pivot

Видим, что в топ-5 среди анализируемых платформ по суммарным продажам с 2013 по 2016 гг. находятся следующие платформы:
1. PS4;
2. PS3;
3. XOne;
4. 3DS;
5. X360.

Проанализируем с помощью графиков, как ведут себя продажи на этих платформах в динамике.

In [ ]:
top_sales_pivot = sales_pivot[:5]
top_sales_pivot = top_sales_pivot.drop('total', axis=1)
top_sales_pivot.T.plot(figsize=(10, 5), grid=True)
plt.title('Динамика глобальных продаж на топ-5 платформах с 2013 по 2016 гг.', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks(ticks=[2013, 2014, 2015, 2016])
plt.xlim(2013, 2016)
plt.legend(title='Платформа', bbox_to_anchor=(1.2, 1), frameon=False)
sns.despine()
plt.show()

Данные за 2016 год можно не брать во внимание по причине того, что они могут быть неполными. Но даже исключая 2016 год из анализа, можно отметить, что 2 из топ-5 платформ увеличивают свои продажи с 2013 года, в то время как остальные 3 - падают.

Посмотрим на более широкую картину: как ведут себя остальные платформы?

In [ ]:
sales_pivot.drop('total', axis=1).T.plot(figsize=(10, 7), grid=True)
plt.title('Динамика глобальных продаж с 2013 по 2016 гг. по платформам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks(ticks=[2013, 2014, 2015, 2016])
plt.xlim(2013, 2016)
plt.legend(title='Платформа', loc='upper right', bbox_to_anchor=(1.2, 1), frameon=False)
sns.despine()
plt.show()

**Большая часть платформ с 2013 года демонстрирует падение продаж.** Самое заметное - у платформ PS3 и X360, на смену которым пришли PS4 и XOne.

**Платформы PS4 и XOne, таким образом, можно отнести к потенциально прибыльным.**

### 3.5. Глобальные продажи игр в разбивке по платформам

In [ ]:
# построим графики "ящик с усами" по глобальным продажам игр в разбивке по платформам

fig, ax = plt.subplots(figsize=(15, 8))
data.boxplot(column=['total_sales'], by='platform', ax=ax)
plt.ylim(0, 2)
fig.suptitle('Диаграммы размаха для глобальных продаж \n с 2013 по 2016 гг. по платформам', fontsize=14)
plt.title('')
plt.xlabel('Платформа', fontsize=12, labelpad=10)
plt.ylabel('Продажи с 2013 по 2016 гг.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

In [ ]:
# построим графики "ящик с усами" по глобальным продажам игр в разбивке по платформам
# со всеми выбросами

fig, ax = plt.subplots(figsize=(15,8))
data.boxplot(column=['total_sales'], by='platform', ax=ax)
fig.suptitle('Диаграммы размаха для глобальных продаж \n с 2013 по 2016 гг. по платформам', fontsize=14)
plt.title('')
plt.xlabel('Платформа', fontsize=12, labelpad=10)
plt.ylabel('Продажи с 2013 по 2016 гг.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

По графикам видим, что наибольшее медианное значение глобальных продаж с 2013 по 2016 гг. демонстрируют платформы **X360, WiiU, XOne**. Далее следуют платформы **PS3 и PS4**, на всех остальных платформах игры медианное значение продаж ниже.

Интересно также наблюдать за степенью разброса значений продаж внутри платформ. Так, например, платформы X360 и PS4 демонстрируют самый большой разброс: несмотря на высокие медианные значения, продажи игр могут быть абсолютно разными. Для сравнения продажи игр на платформах PSP или PS2 имеют гораздо меньший разброс (что связано, скорее всего, с тем, что игры на этих платформах в целом выпускались с 2013 года гораздо реже, поэтому продажи были небольшими и менее вариативными).

Интересно и то, что на самых популярных платформах замечено больше всего выбросов. Вероятно, с 2013 по 2016 год на этих платформах выходили игры, которые завоёвывали огромную популярность у пользователей, что принесло большие продажи.

### 3.6. Влияние отзывов пользователей и критиков на продажи

In [ ]:
# посмотрим, как влияют отзывы пользователей и критиков на продажи внутри платформы PS4

data_ps4 = data[data['platform'] == 'PS4']

print(f"Коэффициент корреляции между переменными: \
{round(data_ps4['user_score'].corr(data_ps4['total_sales']), 2)}")
plt.figure(figsize=(10, 5))
plt.scatter(data_ps4['user_score'], data_ps4['total_sales'])
plt.title('Диаграмма рассеяния для переменных \n user_score и total_sales для платформы PS4',
         fontsize=14, pad=10)
plt.xlabel('Рейтинг пользователей', fontsize=12, labelpad=10)
plt.ylabel('Продажи игр с 2013 по 2016 гг.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

Между переменными наблюдается очень слабая взаимосвязь: **рейтинг пользователей практически никак не связан с продажами игр на платформе.**

Проанализируем аналогичную взаимосвязь между рейтингом критиков и продажами.

In [ ]:
print(f"Коэффициент корреляции между переменными: \
{round(data_ps4['critic_score'].corr(data_ps4['total_sales']), 2)}")

plt.figure(figsize=(10, 5))
plt.scatter(data_ps4['critic_score'], data_ps4['total_sales'])
plt.title('Диаграмма рассеяния для переменных \n critic_score и total_sales для платформы PS4',
         fontsize=14, pad=10)
plt.xlabel('Рейтинг критиков', fontsize=12, labelpad=10)
plt.ylabel('Продажи игр с 2013 по 2016 гг.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

Между переменными *critic_score* и *total_sales* на платформе PS4 наблюдается более сильная взаимосвязь (коэффициент корреляции составляет ~ 0.41). Следовательно, **рейтинг критиков игр на платформе PS4 имеет взаимосвязь среднего уровня с продажами игр с 2013 по 2016 гг.**

Отметим также, что по мере увеличения оценки критиков происходит резкое увеличение продаж, т.е. **продажи демонстрируют скорее экспоненциальный рост по мере увеличения рейтинга критиков**.

Сопоставим наблюдения с остальными платформами. Будет ли наблюдаться та же тенденция: **рейтинг пользователей практически не связан с продажами игр на платформе, в то время как рейтинг критиков имеет более тесную взаимосвязь с продажами?** Ответить на этот вопрос помогут коэффициенты корреляции между переменными *user_score* и *total_sales*, *critic_score* и *total_sales* по каждой переменной.

In [ ]:
# построим матрицу корреляций

corr_matrix = data.groupby('platform')[['critic_score', 'user_score', 'total_sales']].corr()
cm = sns.light_palette("green", as_cmap=True)
corr_matrix = corr_matrix.style.background_gradient(cmap=cm)
corr_matrix

Для 9 из 12 платформ тренд подтверждается: корреляция между рейтингом критиков и продажами более высокая, чем между рейтингом пользователей и продажами. Исключением стали платформы **DS и WiiU**, а также **PS2** по причине того, что данных по критериям недостаточно.

In [ ]:
# изобразим диаграммы рассеяния для топ-5 платформ по парам переменных:
# user_score и total_sales, critic_score и total_sales

for p in data['platform'].unique()[:5]:
    data_platform = data[data['platform'] == p]
    print('-'*30)
    print('Коэффициенты корреляции для платформы', p, ': \n user_score и total_sales:',
          round(data_platform['user_score'].corr(data_platform['total_sales']), 2),
         '\n critic_score и total_sales:',
          round(data_platform['critic_score'].corr(data_platform['total_sales']), 2))
    fig, ax = plt.subplots(1, 2, figsize=(10, 3))
    fig.suptitle(p, fontsize=14)
    ax[0].scatter(data_platform['user_score'], data_platform['total_sales'], alpha=0.5)
    ax[1].scatter(data_platform['critic_score'], data_platform['total_sales'], alpha=0.5)
    ax[0].set(xlabel='Рейтинг пользователей')
    ax[1].set(xlabel='Рейтинг критиков')
    ax[0].set(ylabel='Продажи, млн. шт.')
    ax[1].set(ylabel='Продажи, млн. шт.')
    sns.despine()
    plt.show()

На большинстве из топ-5 платформ подтверждается выявленный тренд: **оценки пользователей практически не связаны с продажами игр, в то время как оценка критиков и продажи связаны положительной взаимосвязью среднего уровня**. Исключением можно назвать платформу 3DS: на ней оценки пользователей также обладают слабой положительной взаимосвязью с продажами игр.

### 3.7. Распределение игр по жанрам

In [ ]:
genre_pivot = (data
               .pivot_table(
                   index='genre',
                   values='total_sales',
                   aggfunc=['sum', 'median'])
)
genre_pivot.columns=['sum', 'median']
genre_pivot = genre_pivot.sort_values(by='sum', ascending=False)
genre_pivot

In [ ]:
genre_pivot = genre_pivot.sort_values(by='median', ascending=False)
genre_pivot

Наибольшие суммарные продажи с 2013 по 2016 гг. демонстрировали игры жанров **Action** и **Shooter**, наименее популярны игры жанров **Strategy** и **Puzzle**.

Однако при сравнении медианных продаж картина меняется: лидерство удерживают игры жанра **Shooter**, а вот **Action** - существенно ниже в рейтинге.

Для более детального изучения данных посмотрим на диаграммы размаха глобальных продаж по жанрам.

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
data.boxplot(column=['total_sales'], by='genre', ax=ax, grid=False)
fig.suptitle('Диаграммы размаха для глобальных продаж по жанрам', fontsize=14)
plt.title('')
plt.xlabel('Жанр', fontsize=12, labelpad=10)
plt.ylabel('Продажи с 2013 по 2016 гг.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

На диаграммах размаха видно: суммарные продажи игр жанра Action больше всех остальных из-за наличия большого количества выбросов - в частности, за последние 3 года было 3 игры с продажами более 10 млн. копий, а одна игра - более 20 млн. копий.

In [ ]:
# посмотрим, какая игра жанра Action набрала наибольшее кол-во продаж за последние 3 года

data[data['genre'] == 'Action'].max()

In [ ]:
# увеличим масштаб диаграмм размаха, чтобы лучше увидеть разброс значений
# по продажам игр в разбивке по жанрам

fig, ax = plt.subplots(figsize=(15, 8))
data.boxplot(column=['total_sales'], by='genre', ax=ax, grid=False)
fig.suptitle('Диаграммы размаха для глобальных продаж по жанрам', fontsize=14)
plt.ylim(0, 4)
plt.title('')
plt.xlabel('Жанр', fontsize=12, labelpad=10)
plt.ylabel('Продажи с 2013 по 2016 гг.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

На основе графиков можно сделать вывод: **более устойчиво выглядят продажи игр жанров Shooter, Sports и Platform**. Проверим, соблюдается ли аналогичный тренд при анализе продаж в динамике.

In [ ]:
# посмотрим, как вели себя глобальные продажи игр по жанрам за последние 3 года

genre_pivot = (data
               .pivot_table(
                   index='genre',
                   values='total_sales',
                   columns='year_of_release',
                   aggfunc='sum')
)

genre_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика глобальных продаж с 2013 по 2016 гг. по жанрам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Жанр', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

In [ ]:
# проверим аналогичную динамику медианных продаж

genre_pivot_median = (data
                      .pivot_table(
                          index='genre',
                          values='total_sales',
                          columns='year_of_release',
                          aggfunc='median')
)

genre_pivot_median.T.plot(figsize=(10, 7))
fig.suptitle('Динамика медианных продаж с 2013 по 2016 гг. по жанрам', fontsize=14)
plt.title('')
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Жанр', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Несмотря на своё лидерство, **с 2013 года игры жанра Action теряют свои позиции на глобальном рынке**.

**Продажи игр жанра Shooter выглядят более стабильно** как с позиции медианных продаж с 2013 по 2016 гг., так и с точки зрения суммарных продаж игр за аналогичный период. Жанры Sports и Platform также выглядят привлекательно с точки зрения медианных продаж, однако мы не можем быть уверены в их стабильности.

### Выводы по итогам исследовательского анализа

**Ключевые выводы по итогам исследовательского анализа:**
1. **пик продаж компьютерных игр интернет-магазина пришёлся на период с 2005 по 2011 года, максимум - в 2009 году. Начиная с этого же года глобальные продажи компании стремительно снижаются**;
2. в "топе" глобальных продаж за последние 3 года (2013-2016 гг.) находятся платформы **PS4, PS3, XOne, 3DS и X360**;
3. **уверенный рост на глобальном рынке за последние 3 года демонстрируют 2 платформы: PS4 и XOne**. Первые игры на них продавались в 2013 году, и на протяжении двух лет темпы роста продаж игр на PS4 и XOne позволили перегнать "старые" платформы;
4. практически на всех платформах не наблюдается взаимосвязи между оценкой пользователей и продажами игр на платформе. **Более сильно на продажи влияют оценки критиков, при этом рост продаж имеет скорее экспоненциальный характер (по мере роста оценок критиков сила связи с продажами также увеличивается)**;
5. **наибольшие суммарные продажи за последние 3 года принесли игры жанра Action, однако с 2013 года они теряют свои позиции на глобальном рынке**. Более стабильно выглядят игры жанра **Shooter**;
6. **средний срок "жизни" игровой платформы составляет 7 лет**, при этом для топ-6 платформ этот показатель существенно выше - 11.5 лет.

<a id='step4'></a>
## 4. Портреты пользователей

Составим портрет пользователя для каждого из представленных регионов (NA, EU, JP). Определим:
1. топ-5 игровых платформ;
2. топ-5 жанров;
3. влияние рейтинга ESRB на продажи.

### 4.1. Северная Америка (NA)

#### 4.1.1. Топ-5 платформ в Северной Америке

In [ ]:
# создадим отдельный датафрейм для игр, проданных в Северной Америке

data_na = data.drop(['eu_sales', 'jp_sales', 'other_sales'], axis=1)
data_na.head()

In [ ]:
data_na_platform_pivot = (data_na
 .pivot_table(index='platform', values='na_sales', aggfunc='sum')
 .sort_values(by='na_sales', ascending=False)
)

data_na_platform_pivot['sales_share'] = (round(data_na_platform_pivot['na_sales'] / 
                                         data_na_platform_pivot['na_sales'].sum() * 100, 1))
data_na_platform_pivot.head()

Наибольшую долю продаж в Северной Америке с 2013 по 2016 гг. составляли игры на платформе **PS4 (24.8% от всех продаж в регионе)**. В пятёрку наиболее популярных платформ в регионе также входят платформы **XOne (21.3%), X360 (18.7%), PS3 (14.5%) и 3DS (8.7%)**.

In [ ]:
# дополнительно проверим, как ведут себя продажи игр 
# на платформах в Северной Америке в динамике

na_platform_pivot = data_na.pivot_table(
    index='platform',
    values='na_sales',
    columns='year_of_release',
    aggfunc='sum'
)

na_platform_pivot['total'] = na_platform_pivot.sum(axis=1)
na_platform_pivot = na_platform_pivot.sort_values(by='total', ascending=False)
na_platform_pivot = na_platform_pivot.drop('total', axis=1)
na_platform_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика продаж в Северной Америке \n с 2013 по 2016 гг. по платформам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Платформа', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Видим следующее: **помимо лидерства в суммарных продажах за 2013-2016 гг., платформы PS4 и XOne демонстрируют рост продаж и в динамике**.

**Продажи игр на платформах X360 и PS3 на рынке Северной Америки с 2013 года постоянно снижаются**.

#### 4.1.2. Топ-5 жанров в Северной Америке

In [ ]:
data_na_genre_pivot = (data_na
 .pivot_table(index='genre', values='na_sales', aggfunc='sum')
 .sort_values(by='na_sales', ascending=False)
)

data_na_genre_pivot['sales_share'] = (round(data_na_genre_pivot['na_sales'] / 
                                         data_na_genre_pivot['na_sales'].sum() * 100, 1))
data_na_genre_pivot.head()

Больше всего в Северной Америке с 2013 по 2016 гг. популярностью пользовались игры жанра **Action (28.8% всех продаж в Северной Америке в соответствующий период)**.

В пятёрку наиболее популярных жанров также вошли: **Shooter (25.1%), Sports (14.9%), Role-Playing (10.6%) и Misc (6.3%)**.

In [ ]:
# дополнительно проверим, как ведут себя продажи игр 
# разных жанров в Северной Америке в динамике

na_genre_pivot = data_na.pivot_table(
    index='genre',
    values='na_sales',
    columns='year_of_release',
    aggfunc='sum'
)

na_genre_pivot['total'] = na_genre_pivot.sum(axis=1)
na_genre_pivot = na_genre_pivot.sort_values(by='total', ascending=False)
na_genre_pivot = na_genre_pivot.drop('total', axis=1)

na_genre_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика продаж в Северной Америке \n с 2013 по 2016 гг. по жанрам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Жанр', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Общая тенденция продаж по жанрам в Северной Америке с 2013 по 2016 гг. соответствует глобальному тренду: **игры жанра Action теряют позиции, в то время как игры жанров Shooter, Sports и Role-Playing постепенно приходят им на замену**.

In [ ]:
# дополнительно проверим, как ведут себя медианные продажи игр 
# разных жанров в Северной Америке в динамике

na_genre_pivot_median = data_na.pivot_table(
    index='genre',
    values='na_sales',
    columns='year_of_release',
    aggfunc='median'
)

na_genre_pivot_median['total'] = na_genre_pivot_median.sum(axis=1)
na_genre_pivot_median = na_genre_pivot_median.sort_values(by='total', ascending=False)
na_genre_pivot_median = na_genre_pivot_median.drop('total', axis=1)

na_genre_pivot_median.T.plot(figsize=(10, 7))
plt.title('Динамика медианных продаж в Северной Америке \n с 2013 по 2016 гг. по жанрам',
         fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Жанр', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Продажи игр в Северной Америке по жанрам подтверждают общий тренд: более устойчиво выглядят игры жанра **Shooter**.

#### 4.1.3. Влияние рейтинга ESRB на продажи игр в Северной Америке

In [ ]:
# построим диаграммы размаха переменной продаж в Северной Америке для каждого рейтинга

fig, ax = plt.subplots(figsize=(15, 8))
data_na.boxplot(column=['na_sales'], by='rating', ax=ax, grid=False)
plt.ylim(0, 2)
fig.suptitle('Диаграммы размаха для продаж в Северной Америке по рейтингу', fontsize=14)
plt.title('')
plt.xlabel('Рейтинг', fontsize=12, labelpad=10)
plt.ylabel('Продажи с 2013 по 2016 гг.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

**Наибольшим спросом в Северной Америке с 2013 по 2016 гг. пользовались игры с рейтингом M (17+).** Посмотрим, сколько всего игр было продано в Северной Америке в разбивке по рейтингу.

In [ ]:
na_sales_by_rating = (data_na
 .pivot_table(index='rating', values='na_sales', aggfunc='sum')
 .sort_values(by='na_sales', ascending=False)
)

na_sales_by_rating['na_sales_share'] = (round(na_sales_by_rating['na_sales'] / 
                                     na_sales_by_rating['na_sales'].sum() * 100, 1))
na_sales_by_rating

**Доля игр рейтинга M (17+), проданных в Северной Америке с 2013 по 2016 гг., составляет 37.7%. В перечень наиболее популярных игр также вошли игры рейтинга E (0+; 18.1% продаж), E10+ (10+; 12.4%) и T (13+; 11.4%).**

20.4% продаж за актуальный период составляют игры, изданные не в США или Канаде (NonNA) и, как следствие, не получившие рейтинг ESRB.

In [ ]:
# дополнительно проверим, как ведут себя продажи игр 
# разного рейтинга в Северной Америке в динамике

na_rating_pivot = data_na.pivot_table(
    index='rating',
    values='na_sales',
    columns='year_of_release',
    aggfunc='sum'
)

na_rating_pivot['total'] = na_rating_pivot.sum(axis=1)
na_rating_pivot = na_rating_pivot.sort_values(by='total', ascending=False)
na_rating_pivot = na_rating_pivot.drop('total', axis=1)

na_rating_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика продаж в Северной Америке \n с 2013 по 2016 гг. по рейтингу', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Рейтинг', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

In [ ]:
data_na['rating'].unique()

В разбивке по рейтингам заметно **существенное падение продаж игр рейтинга M (17+)**.

Интересное наблюдение: **с 2013 года в Северной Америке наблюдается небольшой рост продаж игр, произведённых не в США и не в Канаде** - это можно понять по увеличению продаж игр с установленной заглушкой NonNA.

In [ ]:
# дополнительно проверим, как ведут себя медианные продажи игр 
# разного рейтинга в Северной Америке в динамике

na_rating_pivot_median = data_na.pivot_table(
    index='rating',
    values='na_sales',
    columns='year_of_release',
    aggfunc='median'
)

na_rating_pivot_median['total'] = na_rating_pivot_median.sum(axis=1)
na_rating_pivot_median = na_rating_pivot_median.sort_values(by='total', ascending=False)
na_rating_pivot_median = na_rating_pivot_median.drop('total', axis=1)

na_rating_pivot_median.T.plot(figsize=(10, 7))
plt.title('Динамика медианных продаж в Северной Америке \n с 2013 по 2016 гг. по рейтингу', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Рейтинг', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Несмотря на падение суммарных продаж, **игры жанра M выглядят более устойчиво с точки зрения медианных продаж**.

Медианные продажи игр не из США или Америки находятся на последнем месте. Учитывая рост суммарных продаж таких игр, можно сделать вывод, что **с 2013 года в Северную Америку "зашло" гораздо больше игр из других стран, однако их продажи нельзя считать успешными**. Рост продаж происходит из-за увеличения количества самих игр, а не продаж каждой отдельной игры.

#### 4.1.4. Портрет пользователя из Северной Америки

Исходя из анализа, можем сформировать черты, которые характеризуют **портрет среднестатистического пользователя компьютерных игр в Северной Америке**:
1. приобретает игры для платформ PS4 или XOne;
2. жанр игры - Shooter;
3. рейтинг игры - M (17+).

### 4.2. Европа (EU)

#### 4.2.1. Топ-5 платформ в Европе

In [ ]:
# создадим отдельный датафрейм для игр, проданных в Европе

data_eu = data.drop(['na_sales', 'jp_sales', 'other_sales'], axis=1)
data_eu.head()

In [ ]:
data_eu_platform_pivot = (data_eu
 .pivot_table(index='platform', values='eu_sales', aggfunc='sum')
 .sort_values(by='eu_sales', ascending=False)
)

data_eu_platform_pivot['sales_share'] = (round(data_eu_platform_pivot['eu_sales'] / 
                                         data_eu_platform_pivot['eu_sales'].sum() * 100, 1))
data_eu_platform_pivot.head()

Наибольшую долю продаж в Европе с 2013 по 2016 гг. составляли игры на платформе **PS4 (36.0% от всех продаж в регионе)**. В пятёрку наиболее популярных платформ в регионе также входят платформы **PS3 (17.3%), XOne (13.2%), X360 (10.8%) и 3DS (7.9%)**.

In [ ]:
# дополнительно проверим, как ведут себя продажи игр 
# на платформах в Европе в динамике

eu_platform_pivot = data_eu.pivot_table(
    index='platform',
    values='eu_sales',
    columns='year_of_release',
    aggfunc='sum'
)

eu_platform_pivot['total'] = eu_platform_pivot.sum(axis=1)
eu_platform_pivot = eu_platform_pivot.sort_values(by='total', ascending=False)
eu_platform_pivot = eu_platform_pivot.drop('total', axis=1)

eu_platform_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика продаж в Европе \n с 2013 по 2016 гг. по платформам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Платформа', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

**Рост популярности игр на платформе PS4 в Европе оказался более ярковыраженным, чем в Северной Америке.** Тем не менее, тенденция осталась той же: **на смену платформам PS3 и X360 приходят платформы PS4 и XOne**.

In [ ]:
# дополнительно проверим, как ведут себя медианные продажи игр 
# на платформах в Европе в динамике

eu_platform_pivot_median = data_eu.pivot_table(
    index='platform',
    values='eu_sales',
    columns='year_of_release',
    aggfunc='median'
)

eu_platform_pivot_median['total'] = eu_platform_pivot_median.sum(axis=1)
eu_platform_pivot_median = eu_platform_pivot_median.sort_values(by='total', ascending=False)
eu_platform_pivot_median = eu_platform_pivot_median.drop('total', axis=1)

eu_platform_pivot_median.T.plot(figsize=(10, 7))
plt.title('Динамика медианных продаж в Европе \n с 2013 по 2016 гг. по платформам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Платформа', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

In [ ]:
# посмотрим на динамику общих продаж в Европе с 2013 по 2016 гг.

data_eu.groupby('year_of_release')['total_sales'].sum().plot(figsize=(10, 5))
plt.title('Динамика продаж в Европе с 2013 по 2016 гг.', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
sns.despine()
plt.show()

**Продажи игр в Европе за последние 3 года постоянно снижаются**, однако в разбивке по платформам можно заметить интересную тенденцию: несмотря на всеобщее падение медианных продаж, суммарные продажи на некоторых платформах (PS4, XOne) увеличиваются. Это говорит об **увеличении количества игр, производимых и продаваемых на данных платформах**.

#### 4.2.2. Топ-5 жанров в Европе

In [ ]:
data_eu_genre_pivot = (data_eu
 .pivot_table(index='genre', values='eu_sales', aggfunc='sum')
 .sort_values(by='eu_sales', ascending=False)
)

data_eu_genre_pivot['sales_share'] = (round(data_eu_genre_pivot['eu_sales'] / 
                                         data_eu_genre_pivot['eu_sales'].sum() * 100, 1))
data_eu_genre_pivot.head()

Больше всего в Европе с 2013 по 2016 гг. популярностью пользовались игры жанра **Action (30.1% всех продаж в Европе в соответствующий период)**.

В пятёрку наиболее популярных жанров также вошли: **Shooter (22.4%), Sports (15.4%), Role-Playing (9.4%) и Racing (5.1%)**.

Отметим, что рейтинг лучших жанров в Европе практически полностью совпадает с аналогичным рейтингом в Северной Америке - кроме того, очень близки между собой и показатели доли продаж.

In [ ]:
# дополнительно проверим, как ведут себя продажи игр 
# разных жанров в Европе в динамике

eu_genre_pivot = data_eu.pivot_table(
    index='genre',
    values='eu_sales',
    columns='year_of_release',
    aggfunc='sum'
)

eu_genre_pivot['total'] = eu_genre_pivot.sum(axis=1)
eu_genre_pivot = eu_genre_pivot.sort_values(by='total', ascending=False)
eu_genre_pivot = eu_genre_pivot.drop('total', axis=1)

eu_genre_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика продаж в Европе \n с 2013 по 2016 гг. по жанрам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Жанр', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Как и в Северной Америке, **продажи игр жанра Action в Европе демонстрируют существенное падение**. Оно **частично компенсируется жанрами Shooter, Sports и Role-Playing**.

In [ ]:
# дополнительно проверим, как ведут себя медианные продажи игр 
# разных жанров в Европе в динамике

eu_genre_pivot_median = data_eu.pivot_table(
    index='genre',
    values='eu_sales',
    columns='year_of_release',
    aggfunc='median'
)

eu_genre_pivot_median['total'] = eu_genre_pivot_median.sum(axis=1)
eu_genre_pivot_median = eu_genre_pivot_median.sort_values(by='total', ascending=False)
eu_genre_pivot_median = eu_genre_pivot_median.drop('total', axis=1)

eu_genre_pivot_median.T.plot(figsize=(10, 7))
plt.title('Динамика медианных продаж в Европе \n с 2013 по 2016 гг. по жанрам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Жанр', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

В Европе наблюдается та же тенденция медианных продаж с точки зрения жанров: **наиболее перспективными выглядят игры жанра Shooter**.

Игры жанров Sports и Role-Playing, несмотря на большие суммарные продажи, не демонстрируют высокие медианные продажи.

#### 4.2.3. Влияние рейтинга ESRB на продажи игр в Европе

In [ ]:
# построим диаграммы размаха переменной продаж в Европе для каждого рейтинга

fig, ax = plt.subplots(figsize=(15, 8))
data_eu.boxplot(column=['eu_sales'], by='rating', ax=ax, grid=False)
plt.ylim(0, 2)
fig.suptitle('Диаграммы размаха для продаж в Европе по рейтингу', fontsize=14)
plt.title('')
plt.xlabel('Рейтинг', fontsize=12, labelpad=10)
plt.ylabel('Продажи с 2013 по 2016 гг.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

Аналогично Северной Америке, **наибольшей популярностью в Европе с 2013 по 2016 гг. пользовались игры рейтинга M (17+)**. Посмотрим на количество проданных игр каждого рейтинга в Европе в этот период.

In [ ]:
eu_sales_by_rating = (data_eu
 .pivot_table(index='rating', values='eu_sales', aggfunc='sum')
 .sort_values(by='eu_sales', ascending=False)
)

eu_sales_by_rating['eu_sales_share'] = (round(eu_sales_by_rating['eu_sales'] / 
                                     eu_sales_by_rating['eu_sales'].sum() * 100, 1))
eu_sales_by_rating

**Общие черты** с аналогичным распределением в Северной Америке:
1. **наиболее популярные игры, проданные в Европе с 2013 по 2016 гг., имели рейтинг M с долей продаж 37.0% от всех игр**, проданных в Европе в указанный период (для сравнения: аналогичная доля продаж в Северной Америке составила 37.7%);
2. **доли игр рейтинга E10+ схожи и находятся на 4 месте (10.9% vs. 12.4)**;
3. **на последнем месте по продажам - игры с рейтингом T (10.7% vs. 11.4%)**.

**Отличия:**
1. **игры, произведённые не в США или Канаде, пользуются в Европе чуть меньшей популярностью (20.1% vs. 20.4%)**, занимая 3 место по продажам (в Северной Америке - 2 место);
2. **игры рейтинга E более популярны, чем в Северной Америке (21.3% vs. 18.1%)**.

In [ ]:
# дополнительно проверим, как ведут себя продажи игр 
# разного рейтинга в Европе в динамике

eu_rating_pivot = data_eu.pivot_table(
    index='rating',
    values='eu_sales',
    columns='year_of_release',
    aggfunc='sum'
)

eu_rating_pivot['total'] = eu_rating_pivot.sum(axis=1)
eu_rating_pivot = eu_rating_pivot.sort_values(by='total', ascending=False)
eu_rating_pivot = eu_rating_pivot.drop('total', axis=1)

eu_rating_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика продаж в Европе \n с 2013 по 2016 гг. по рейтингу', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Рейтинг', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Картина выглядит очень похоже на Северную Америку: **игры всех возрастных рейтингов стремительно теряют популярность, при этом наибольшее падение - у игр рейтинга M (17+)**.

**Суммарные продажи игр, произведённых не в США или Канаде, однако, также демонстрируют рост.**

In [ ]:
# дополнительно проверим, как ведут себя медианные продажи игр 
# разного рейтинга в Европе в динамике

eu_rating_pivot_median = data_eu.pivot_table(
    index='rating',
    values='eu_sales',
    columns='year_of_release',
    aggfunc='median'
)

eu_rating_pivot_median['total'] = eu_rating_pivot_median.sum(axis=1)
eu_rating_pivot_median = eu_rating_pivot_median.sort_values(by='total', ascending=False)
eu_rating_pivot_median = eu_rating_pivot_median.drop('total', axis=1)

eu_rating_pivot_median.T.plot(figsize=(10, 7))
plt.title('Динамика медианных продаж в Европе \n с 2013 по 2016 гг. по рейтингу', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Рейтинг', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

По медианным продажам **наибольшее падение заметно у игр рейтинга E10+, хотя уже на данный момент в 2016 г. они опередили продажи игр жанра M**.

В этом смысле **более стабильно выглядят продажи игр рейтинга T**, демонстрируя стабильные медианные продажи, однако пока нельзя назвать игры с этим рейтингом перспективными.

#### 4.2.4. Портрет пользователя из Европы

Исходя из анализа, можем сформировать черты, которые характеризуют **портрет среднестатистического пользователя компьютерных игр в Европе**:
1. приобретает игры для платформы PS4;
2. жанр игры - Shooter;
3. рейтинг игры - M (17+) или E10+ (10+).

### 4.3. Япония (JP)

#### 4.3.1. Топ-5 платформ в Японии

In [ ]:
# создадим отдельный датафрейм для игр, проданных в Японии

data_jp = data.drop(['na_sales', 'eu_sales', 'other_sales'], axis=1)
data_jp.head()

In [ ]:
data_jp_platform_pivot = (data_jp
 .pivot_table(index='platform', values='jp_sales', aggfunc='sum')
 .sort_values(by='jp_sales', ascending=False)
)

data_jp_platform_pivot['sales_share'] = (round(data_jp_platform_pivot['jp_sales'] / 
                                         data_jp_platform_pivot['jp_sales'].sum() * 100, 1))
data_jp_platform_pivot.head()

Наибольшую долю продаж в Японии с 2013 по 2016 гг. составляли игры на платформе **3DS (48.2% от всех продаж в регионе)**. Это неудивительно, т.к. сама платформа была создана именно в Японии.

В пятёрку наиболее популярных платформ в регионе также входят платформы **PS3 (16.6%), PSV (13.2%), PS4 (11.3%) и WiiU (7.7%)**.

In [ ]:
# дополнительно проверим, как ведут себя продажи игр на платформах в Японии в динамике

jp_platform_pivot = data_jp.pivot_table(
    index='platform',
    values='jp_sales',
    columns='year_of_release',
    aggfunc='sum'
)

jp_platform_pivot['total'] = jp_platform_pivot.sum(axis=1)
jp_platform_pivot = jp_platform_pivot.sort_values(by='total', ascending=False)
jp_platform_pivot = jp_platform_pivot.drop('total', axis=1)

jp_platform_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика продаж в Японии \n с 2013 по 2016 гг. по платформам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Платформа', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Несмотря на то, что игры на платформе 3DS составляют львиную долю всех продаж в регионе, **продажи игр на этой платформе постоянно падают с 2013 года**.

**Стоит обратить внимание на платформы PS4, PSV и WiiU, которые демонстрируют либо рост (PS4), либо стабильность продаж (PSV и WiiU) с 2013 по 2015 гг.**

In [ ]:
# дополнительно проверим, как ведут себя медианные продажи игр 
# на платформах в Японии в динамике

jp_platform_pivot_median = data_jp.pivot_table(
    index='platform',
    values='jp_sales',
    columns='year_of_release',
    aggfunc='median'
)

jp_platform_pivot_median['total'] = jp_platform_pivot_median.sum(axis=1)
jp_platform_pivot_median = jp_platform_pivot_median.sort_values(by='total', ascending=False)
jp_platform_pivot_median = jp_platform_pivot_median.drop('total', axis=1)

jp_platform_pivot_median.T.plot(figsize=(10, 7))
plt.title('Динамика медианных продаж в Японии \n с 2013 по 2016 гг. по платформам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Платформа', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

**Платформа 3DS является абсолютным победителем** как по суммарным, так и по медианным продажам, поэтому именно **она является наиболее перспективной на японском рынке**.

#### 4.3.2. Топ-5 жанров в Японии

In [ ]:
data_jp_genre_pivot = (data_jp
 .pivot_table(index='genre', values='jp_sales', aggfunc='sum')
 .sort_values(by='jp_sales', ascending=False)
)

data_jp_genre_pivot['sales_share'] = (round(data_jp_genre_pivot['jp_sales'] / 
                                         data_jp_genre_pivot['jp_sales'].sum() * 100, 1))
data_jp_genre_pivot.head()

Больше всего в Японии с 2013 по 2016 гг. популярностью пользовались игры жанра **Role-Playing (36.3% всех продаж в Японии в соответствующий период)**. В пятёрку наиболее популярных жанров также вошли: **Action (28.8%), Misc (6.5%), Fighting (5.4%) и Shooter (4.7%)**.

Рейтинг лучших жанров в Японии отличается от топ-5 жанров в Северной Америке и Европе. Жанр **Action** находится среди лидеров, но в перечне появилось несколько новых жанров (**Fighting и Misc**), а первое место заняли игры жанра **Role-Playing**, популярность которых гораздо ниже среди пользователей Северной Америки и Европы.

In [ ]:
# дополнительно проверим, как ведут себя продажи игр 
# разных жанров в Японии в динамике

jp_genre_pivot = data_jp.pivot_table(
    index='genre',
    values='jp_sales',
    columns='year_of_release',
    aggfunc='sum'
)

jp_genre_pivot['total'] = jp_genre_pivot.sum(axis=1)
jp_genre_pivot = jp_genre_pivot.sort_values(by='total', ascending=False)
jp_genre_pivot = jp_genre_pivot.drop('total', axis=1)

jp_genre_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика продаж в Японии \n с 2013 по 2016 гг. по жанрам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Жанр', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Как и с платформами, **японский рынок компьютерных игр отличается от рынков Северной Америки и Европы в разрезе жанров**.

**До 2014 года самым популярным жанром в регионе был Role-Playing**, однако **в 2015 году картина существенно изменилась: на первое место вышли игры жанра Action, а жанр Role-Playing резко упал в популярности**.

В 2014 году также на третье место по продажам в Японии вырвались игры жанра Fighting, но уже в 2015 году оно было занято играми жанра Shooter.

In [ ]:
# дополнительно проверим, как ведут себя медианные продажи игр 
# разных жанров в Японии в динамике

jp_genre_pivot_median = data_jp.pivot_table(
    index='genre',
    values='jp_sales',
    columns='year_of_release',
    aggfunc='median'
)

jp_genre_pivot_median['total'] = jp_genre_pivot_median.sum(axis=1)
jp_genre_pivot_median = jp_genre_pivot_median.sort_values(by='total', ascending=False)
jp_genre_pivot_median = jp_genre_pivot_median.drop('total', axis=1)

jp_genre_pivot_median.T.plot(figsize=(10, 7))
plt.title('Динамика медианных продаж в Японии \n с 2013 по 2016 гг. по жанрам', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Жанр', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Несмотря на всплеск популярности игр жанра Action в Японии в 2015 году, подобные игры не могут похвастаться высокими медианными продажами. Напротив, **игры жанра Role-Playing выглядят более стабильно**.

#### 4.3.3. Влияние рейтинга ESRB на продажи игр в Японии

In [ ]:
# построим диаграммы размаха переменной продаж в Японии для каждого рейтинга

fig, ax = plt.subplots(figsize=(15, 8))
data_jp.boxplot(column=['jp_sales'], by='rating', ax=ax, grid=False)
plt.ylim(0, 0.3)
fig.suptitle('Диаграммы размаха для продаж в Японии по рейтингу', fontsize=14)
plt.title('')
plt.xlabel('Рейтинг', fontsize=12, labelpad=10)
plt.ylabel('Продажи с 2013 по 2016 гг.', fontsize=12, labelpad=10)
sns.despine()
plt.show()

In [ ]:
jp_sales_by_rating = (data_jp
 .pivot_table(index='rating', values='jp_sales', aggfunc='sum')
 .sort_values(by='jp_sales', ascending=False)
)

jp_sales_by_rating['jp_sales_share'] = (round(jp_sales_by_rating['jp_sales'] / 
                                        jp_sales_by_rating['jp_sales'].sum() * 100, 1))
jp_sales_by_rating

In [ ]:
# дополнительно проверим, как ведут себя продажи игр 
# разного рейтинга в Японии в динамике

jp_rating_pivot = data_jp.pivot_table(
    index='rating',
    values='jp_sales',
    columns='year_of_release',
    aggfunc='sum'
)

jp_rating_pivot['total'] = jp_rating_pivot.sum(axis=1)
jp_rating_pivot = jp_rating_pivot.sort_values(by='total', ascending=False)
jp_rating_pivot = jp_rating_pivot.drop('total', axis=1)

jp_rating_pivot.T.plot(figsize=(10, 7))
plt.title('Динамика продаж в Японии \n с 2013 по 2016 гг. по рейтингу', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Рейтинг', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Любопытное наблюдение: **более половины суммарных продаж игр в Японии составляют игры, произведённые не в США или Канаде (60.4%)**. Продажи игр с другим рейтингом существенно ниже.

In [ ]:
# дополнительно проверим, как ведут себя медианные продажи игр 
# разного рейтинга в Японии в динамике

jp_rating_pivot_median = data_jp.pivot_table(
    index='rating',
    values='jp_sales',
    columns='year_of_release',
    aggfunc='median'
)

jp_rating_pivot_median['total'] = jp_rating_pivot_median.sum(axis=1)
jp_rating_pivot_median = jp_rating_pivot_median.sort_values(by='total', ascending=False)
jp_rating_pivot_median = jp_rating_pivot_median.drop('total', axis=1)

jp_rating_pivot_median.T.plot(figsize=(10, 7))
plt.title('Динамика медианных продаж в Японии \n с 2013 по 2016 гг. по рейтингу', fontsize=14, pad=10)
plt.xlabel('Год', fontsize=12, labelpad=10)
plt.ylabel('Продажи, млн. шт.', fontsize=12, labelpad=10)
plt.xticks([2013, 2014, 2015, 2016])
plt.legend(title='Рейтинг', frameon=False, bbox_to_anchor=(1.2, 1))
sns.despine()
plt.show()

Несмотря на значительное превосходство в 2013 году, уже в 2014 году игры с рейтингом T сравнялись по продажам с играми, у которых рейтинга нет, а в 2016 году, даже основываясь на неполных данных, игры, произведённые не в США или Канаде, выглядят абсолютным победителем.

**В Японии, таким образом, гораздо большей популярностью и лучшими перспективами обладают игры, произведённые не в США или Канаде**.

In [ ]:
# объединим полученные таблицы по регионам

(na_sales_by_rating
 .join([eu_sales_by_rating, jp_sales_by_rating])
)

Картина продаж компьютерных игр в Японии с 2013 по 2016 гг. существенно отличается от продаж в Северной Америке и Европе за аналогичный период:
1. **первое место по продажам в Японии заняли игры без рейтинга, т.е. произведённые не в США или Канаде (60.4%);
2. **на втором месте - игры с рейтигом T (13+; 14.6% продаж)**;
3. **игры с рейтингом M (17+) - только на четвёртом месте с долей продаж 10.0%**, что практически в 4 раза ниже аналогичного показателя в Северной Америке и Европе (37.7% и 37.0% соответственно).

Как видим из анализа, **в Японии гораздо большую долю продаж составляют игры, произведённые не в США или Канаде**. Доля продаж таких игр в Северной Америке и Европе меньше примерно в 3 раза.

#### 4.3.4. Портрет пользователя из Японии

Исходя из анализа, можем сформировать черты, которые характеризуют **портрет среднестатистического пользователя компьютерных игр в Японии**:
1. приобретает игры для платформы 3DS;
2. жанр игры - Role-Playing;
3. рейтинг игры - без рейтинга (т.е. игры произведены не в США или Канаде).

### Результаты анализа портретов пользователей по регионам

In [ ]:
# отобразим визуально разницу распределения продаж игр
# по платформам в разбивке по регионам

# NA
data_na_platform_pivot = data_na_platform_pivot.reset_index()

na_other_sales = pd.Series(['other', 
                            data_na_platform_pivot.loc[5:10, 'na_sales'].sum(),
                            data_na_platform_pivot.loc[5:10, 'sales_share'].sum()],
                            index=['platform', 'na_sales', 'sales_share'])

data_na_platform_pivot = data_na_platform_pivot.append(na_other_sales, ignore_index=True)
data_na_platform_pivot = data_na_platform_pivot.drop(data_na_platform_pivot.index[5:11])


# EU
data_eu_platform_pivot = data_eu_platform_pivot.reset_index()

eu_other_sales = pd.Series(['other', 
                            data_eu_platform_pivot.loc[5:10, 'eu_sales'].sum(),
                            data_eu_platform_pivot.loc[5:10, 'sales_share'].sum()],
                            index=['platform', 'eu_sales', 'sales_share'])

data_eu_platform_pivot = data_eu_platform_pivot.append(eu_other_sales, ignore_index=True)
data_eu_platform_pivot = data_eu_platform_pivot.drop(data_eu_platform_pivot.index[5:11])

# JP
data_jp_platform_pivot = data_jp_platform_pivot.reset_index()

jp_other_sales = pd.Series(['other', 
                            data_jp_platform_pivot.loc[5:10, 'jp_sales'].sum(),
                            data_jp_platform_pivot.loc[5:10, 'sales_share'].sum()],
                            index=['platform', 'jp_sales', 'sales_share'])

data_jp_platform_pivot = data_jp_platform_pivot.append(jp_other_sales, ignore_index=True)
data_jp_platform_pivot = data_jp_platform_pivot.drop(data_jp_platform_pivot.index[5:11])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 5))
fig.suptitle('Продажи игр по платформам в разных регионах', size=14)

ax[0].pie(data_na_platform_pivot['sales_share'], labels=data_na_platform_pivot['platform'], autopct='%1.1f%%')
ax[0].set_title('Северная Америка')

ax[1].pie(data_eu_platform_pivot['sales_share'], labels=data_eu_platform_pivot['platform'], autopct='%1.1f%%')
ax[1].set_title('Европа')

ax[2].pie(data_jp_platform_pivot['sales_share'], labels=data_jp_platform_pivot['platform'], autopct='%1.1f%%')
ax[2].set_title('Япония')

plt.show()

In [ ]:
# отобразим визуально разницу распределения продаж игр
# по жанрам в разбивке по регионам

# NA
data_na_genre_pivot = data_na_genre_pivot.reset_index()

na_other_sales = pd.Series(['other', 
                            data_na_genre_pivot.loc[5:10, 'na_sales'].sum(),
                            data_na_genre_pivot.loc[5:10, 'sales_share'].sum()],
                            index=['genre', 'na_sales', 'sales_share'])

data_na_genre_pivot = data_na_genre_pivot.append(na_other_sales, ignore_index=True)
data_na_genre_pivot = data_na_genre_pivot.drop(data_na_genre_pivot.index[5:11])


# EU
data_eu_genre_pivot = data_eu_genre_pivot.reset_index()

eu_other_sales = pd.Series(['other', 
                            data_eu_genre_pivot.loc[5:10, 'eu_sales'].sum(),
                            data_eu_genre_pivot.loc[5:10, 'sales_share'].sum()],
                            index=['genre', 'eu_sales', 'sales_share'])

data_eu_genre_pivot = data_eu_genre_pivot.append(eu_other_sales, ignore_index=True)
data_eu_genre_pivot = data_eu_genre_pivot.drop(data_eu_genre_pivot.index[5:11])

# JP
data_jp_genre_pivot = data_jp_genre_pivot.reset_index()

jp_other_sales = pd.Series(['other', 
                            data_jp_genre_pivot.loc[5:10, 'jp_sales'].sum(),
                            data_jp_genre_pivot.loc[5:10, 'sales_share'].sum()],
                            index=['genre', 'jp_sales', 'sales_share'])

data_jp_genre_pivot = data_jp_genre_pivot.append(jp_other_sales, ignore_index=True)
data_jp_genre_pivot = data_jp_genre_pivot.drop(data_jp_genre_pivot.index[5:11])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 5))
fig.suptitle('Продажи игр по жанрам в разных регионах', size=14)

ax[0].pie(data_na_genre_pivot['sales_share'], labels=data_na_genre_pivot['genre'], autopct='%1.1f%%')
ax[0].set_title('Северная Америка')

ax[1].pie(data_eu_genre_pivot['sales_share'], labels=data_eu_genre_pivot['genre'], autopct='%1.1f%%')
ax[1].set_title('Европа')

ax[2].pie(data_jp_genre_pivot['sales_share'], labels=data_jp_genre_pivot['genre'], autopct='%1.1f%%')
ax[2].set_title('Япония')

plt.show()

In [ ]:
# отобразим визуально разницу распределения продаж игр
# по рейтингу в разбивке по регионам

# NA
na_sales_by_rating = na_sales_by_rating.reset_index()

na_other_sales = pd.Series(['other', 
                            na_sales_by_rating.loc[5:10, 'na_sales'].sum(),
                            na_sales_by_rating.loc[5:10, 'na_sales_share'].sum()],
                            index=['rating', 'na_sales', 'na_sales_share'])

na_sales_by_rating = na_sales_by_rating.append(na_other_sales, ignore_index=True)
na_sales_by_rating = na_sales_by_rating.drop(na_sales_by_rating.index[5:11])


# EU
eu_sales_by_rating = eu_sales_by_rating.reset_index()

eu_other_sales = pd.Series(['other', 
                            eu_sales_by_rating.loc[5:10, 'eu_sales'].sum(),
                            eu_sales_by_rating.loc[5:10, 'eu_sales_share'].sum()],
                            index=['rating', 'eu_sales', 'eu_sales_share'])

eu_sales_by_rating = eu_sales_by_rating.append(eu_other_sales, ignore_index=True)
eu_sales_by_rating = eu_sales_by_rating.drop(eu_sales_by_rating.index[5:11])

# JP
jp_sales_by_rating = jp_sales_by_rating.reset_index()

jp_other_sales = pd.Series(['other', 
                            jp_sales_by_rating.loc[5:10, 'jp_sales'].sum(),
                            jp_sales_by_rating.loc[5:10, 'jp_sales_share'].sum()],
                            index=['rating', 'jp_sales', 'jp_sales_share'])

jp_sales_by_rating = jp_sales_by_rating.append(jp_other_sales, ignore_index=True)
jp_sales_by_rating = jp_sales_by_rating.drop(jp_sales_by_rating.index[5:11])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 5))
fig.suptitle('Продажи игр по рейтингу в разных регионах', size=14)

ax[0].pie(na_sales_by_rating['na_sales_share'], labels=na_sales_by_rating['rating'], autopct='%1.1f%%')
ax[0].set_title('Северная Америка')

ax[1].pie(eu_sales_by_rating['eu_sales_share'], labels=eu_sales_by_rating['rating'], autopct='%1.1f%%')
ax[1].set_title('Европа')

ax[2].pie(jp_sales_by_rating['jp_sales_share'], labels=jp_sales_by_rating['rating'], autopct='%1.1f%%')
ax[2].set_title('Япония')

plt.show()

In [ ]:
# на основе проведённого анализа можно сделать сводную таблицу 
# с платформами, жанрами и рейтингами, наиболее популярными в разных регионах

# NA
na_platform = (data_na_platform_pivot[data_na_platform_pivot['na_sales'] == 
                        data_na_platform_pivot['na_sales'].max()]['platform'][0])
na_genre = (data_na_genre_pivot[data_na_genre_pivot['na_sales'] == 
                        data_na_genre_pivot['na_sales'].max()]['genre'][0])
na_rating = (na_sales_by_rating[na_sales_by_rating['na_sales'] == 
                        na_sales_by_rating['na_sales'].max()]['rating'][0])

# EU
eu_platform = (data_eu_platform_pivot[data_eu_platform_pivot['eu_sales'] == 
                        data_eu_platform_pivot['eu_sales'].max()]['platform'][0])
eu_genre = (data_eu_genre_pivot[data_eu_genre_pivot['eu_sales'] == 
                        data_eu_genre_pivot['eu_sales'].max()]['genre'][0])
eu_rating = (eu_sales_by_rating[eu_sales_by_rating['eu_sales'] == 
                        eu_sales_by_rating['eu_sales'].max()]['rating'][0])

# JP
jp_platform = (data_jp_platform_pivot[data_jp_platform_pivot['jp_sales'] == 
                        data_jp_platform_pivot['jp_sales'].max()]['platform'][0])
jp_genre = (data_jp_genre_pivot[data_jp_genre_pivot['jp_sales'] == 
                        data_jp_genre_pivot['jp_sales'].max()]['genre'][0])
jp_rating = (jp_sales_by_rating[jp_sales_by_rating['jp_sales'] == 
                        jp_sales_by_rating['jp_sales'].max()]['rating'][0])

# создадим общую сводную таблицу
pd.DataFrame(
    {'platform': [na_platform, eu_platform, jp_platform],
    'genre': [na_genre, eu_genre, jp_genre],
    'rating': [na_rating, eu_rating, jp_rating]},
    index=(['NA', 'EU', 'JP'])
)

Стоит, однако, брать во внимание и портреты потенциальных пользователей в каждом регионе:

|        | **platform** | **genre**    | **rating** |
|--------|--------------|--------------|------------|
| **NA** | XOne или PS4 | Shooter      | M (17+)    |
| **EU** | PS4          | Shooter      | M (17+)    |
| **JP** | 3DS          | Role-Playing | NonNA      |

После составления портрета пользователей по регионам можно переходить к статистическому анализу данных.

<a id='step5'></a>
## 5. Статистический анализ данных

### 5.1. Гипотеза 1

**Гипотеза 1:** Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.


Для проведения анализа сформулируем нулевую и альтернативную гипотезы.

**H0:** Средний пользовательский рейтинг игр (*user_score*) на платформе Xbox One равен среднему пользовательскому рейтингу игр на платформе PC.

**H1:** Средний пользовательский рейтинг игр (*user_score*) на платформе Xbox One НЕ равен среднему пользовательскому рейтингу игр на платформе PC.


Гипотеза **о равенстве средних** сформулирована для **двух независимых выборок**, а альтернативная гипотеза - двусторонняя. Следовательно, для тестирования гипотез будет логично применить двусторонний t-test для независимых выборок (**ttest_ind**).

In [ ]:
data_xone = data[data['platform'] == 'XOne']
data_pc = data[data['platform'] == 'PC']

alpha = .05

result = st.ttest_ind(data_xone['user_score'], data_pc['user_score'], nan_policy='omit')

print(f"p-value = {round(result.pvalue, 3)}")

if result.pvalue < alpha:
    print('Нулевая гипотеза отклоняется в пользу альтернативной')
else:
    print('Не удалось отклонить нулевую гипотезу')

Исходя из результатов тестирования гипотез, не получилось отклонить нулевую гипотезу. Следовательно, мы не можем исключить, что **средний пользовательский рейтинг игр на платформе Xbox One равен среднему пользовательскому рейтингу игр на платформе PC.**

In [ ]:
# проверим, на каком уровне находится 
# средний пользовательский рейтинг игр на двух платформах в нашей выборке

print('Средний пользовательский рейтинг игр на платформе XOne:', 
      round(data_xone['user_score'].mean(), 2))
print('Средний пользовательский рейтинг игр на платформе PC:', 
      round(data_pc['user_score'].mean(), 2))

### 5.2. Гипотеза 2

**Гипотеза 2:** Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.

**H0:** Средний пользовательский рейтинг (*user_score*) игр жанра Action равен среднему пользовательскому рейтингу игр жанра Sports.

**H1:** Средний пользовательский рейтинг (*user_score*) игр жанра Action НЕ равен среднему пользовательскому рейтингу игр жанра Sports.

Аналогично предыдущей гипотезе, имеет смысл применить двусторонний t-test для её проверки.

In [ ]:
data_action = data[data['genre'] == 'Action']
data_sports = data[data['genre'] == 'Sports']

alpha = .05

result_2 = st.ttest_ind(
    data_action['user_score'], data_sports['user_score'], nan_policy='omit')

print(f"p-value = {result_2.pvalue}")

if result_2.pvalue < alpha:
    print('Нулевая гипотеза отклоняется в пользу альтернативной')
else:
    print('Не удалось отклонить нулевую гипотезу')

На имеющихся данных на 5%-ном уровне значимости **есть основания отвергнуть нулевую гипотезу о равенстве средних значений пользовательских рейтингов (отзывов) жанров Action и Sports в пользу альтернативной гипотезы**.

Проверим средние значения пользовательского рейтинга игр двух жанров и с помощью дополнительного одностороннего теста проверим, в чью пользу наблюдается повышенный рейтинг.

In [ ]:
print('Средний пользовательский рейтинг игр жанра Action:', 
      round(data_action['user_score'].mean(), 2))
print('Средний пользовательский рейтинг игр жанра Sports:', 
      round(data_sports['user_score'].mean(), 2))

In [ ]:
alpha = .05

result_3 = st.ttest_ind(
    data_action['user_score'], data_sports['user_score'],
    nan_policy='omit', alternative='greater')

if result_3.pvalue < alpha:
    print('Нулевая гипотеза отклоняется в пользу альтернативной')
else:
    print('Нулевая гипотеза не отклоняется')

По результатам дополнительного теста было определено, что в генеральной совокупности **пользовательский рейтинг игр жанра Action выше, чем аналогичный показатель для игр жанра Sports**.

## Выводы исследования

Наибольшим потенциалом обладают инвестиции в игры на разных платформах, разных жанров и рейтинга в различных регионах:
1. **в Северной Америке и Европе:**
    1. платформа - PS4;
    2. жанр - Shooter;
    3. рейтинг - M (17+).
    
2. **в Японии**:
    1. платформа - 3DS;
    2. жанр - Role-Playing;
    3. рейтинг - без рейтинга (т.е. игры, произведённые не в США или Канаде).
    
При планировании дальнейшей стратегии компании стоит также учитывать факторы:
1. на продажи игры **могут положительно влиять оценки критиков**, при этом рост продаж имеет скорее экспоненциальный характер (чем больше оценка критиков - тем больше влияние на продажи). А вот **оценки пользователей с продажами практически не связаны**;
2. средний срок жизни платформы - 7 лет, для лучших платформ - 11.5 лет. **Если с момента запуска платформы прошло больше 7 лет, инвестиции в неё будут менее целесообразны**.

**[В начало](#start)**